In [13]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from math import cos,sin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from keras.layers import Dense, Activation
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.metrics import mean_absolute_error
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.utils.vis_utils import plot_model

In [2]:
df = pd.read_csv('cities_minustemps.csv',compression="gzip")
df = df.astype({'City':str, \
                'Latitude':'float32', \
                'Longitude':'float32', \
                'x':'float32','y':'float32', \
                'z':'float32', \
                'Month':'int32', \
                'Year':'int32', \
                'TempMinus1':'float32', 'TempMinus2':'float32','AverageTemperature':'float32'})

label_binarizer = LabelBinarizer()
  
# fitting the column 
# TEMPERATURE to LabelBinarizer
label_binarizer_output = label_binarizer.fit_transform( df['Month'])

df = df.merge(pd.DataFrame(label_binarizer_output,columns=['January','February','March','April','May','June','July','August','September','October','November','December']).astype(bool),left_index=True,right_index=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 27 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Unnamed: 0                     int64  
 1   dt                             object 
 2   AverageTemperature             float32
 3   AverageTemperatureUncertainty  float64
 4   City                           object 
 5   Country                        object 
 6   Latitude                       float32
 7   Longitude                      float32
 8   x                              float32
 9   y                              float32
 10  z                              float32
 11  Month                          int32  
 12  Year                           int32  
 13  TempMinus1                     float32
 14  TempMinus2                     float32
 15  January                        bool   
 16  February                       bool   
 17  March                          bool   
 18  Ap

In [4]:
pca_data_columns = ['Year',
 'x','y','z',
'TempMinus1','TempMinus2',
'January','February','March','April','May','June','July','August','September','October','November','December']
 #'Latitude',
 #'Longitude']

pca_df = df[pca_data_columns+['AverageTemperature']].dropna()

target_columns = ['AverageTemperature']


# create the RFE model and select 10 attributes
#rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=17)
#rfe = rfe.fit(pca_df[pca_data_columns], pca_df[target_columns])
# summarize the selection of the attributes
#print(rfe.support_)
#print(rfe.ranking_)

In [5]:
# Use features from RFE
data_columns = pca_data_columns #list(rfe.feature_names_in_[rfe.support_])

target_columns = ['AverageTemperature']

model_df = df[data_columns+['AverageTemperature']].dropna()

# Grab data and target columns
y = model_df[target_columns]
X = model_df[data_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5376962 entries, 7206987 to 6887963
Data columns (total 18 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Year        int32  
 1   x           float32
 2   y           float32
 3   z           float32
 4   TempMinus1  float32
 5   TempMinus2  float32
 6   January     bool   
 7   February    bool   
 8   March       bool   
 9   April       bool   
 10  May         bool   
 11  June        bool   
 12  July        bool   
 13  August      bool   
 14  September   bool   
 15  October     bool   
 16  November    bool   
 17  December    bool   
dtypes: bool(12), float32(5), int32(1)
memory usage: 225.6 MB


In [7]:
sc = StandardScaler()
X_trains = sc.fit_transform(X_train)
X_tests = sc.transform(X_test)

#tuner.search(X_trains, y_train, epochs=5, validation_data=(X_tests, y_test))
#model = tuner.get_best_models()[0]

# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(18, activation = 'tanh', input_dim = 18))

# Adding the second hidden layer
model.add(Dense(units = 16, activation = 'tanh'))

# Adding the third hidden layer
model.add(Dense(units = 16, activation = 'tanh'))

# Adding the output layer
model.add(Dense(units = 1))

optimizer = Adam() 

model.compile(optimizer = optimizer,loss = 'mean_squared_error')

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                342       
                                                                 
 dense_1 (Dense)             (None, 16)                304       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 935
Trainable params: 935
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_trains, y_train, batch_size = 50, epochs = 5)

Epoch 1/5
107540/107540 [==============================] - 89s 817us/step - loss: 3.4528
Epoch 2/5
107540/107540 [==============================] - 86s 796us/step - loss: 2.3235
Epoch 3/5
107540/107540 [==============================] - 88s 814us/step - loss: 2.3115
Epoch 4/5
107540/107540 [==============================] - 86s 800us/step - loss: 2.3047
Epoch 5/5
107540/107540 [==============================] - 85s 794us/step - loss: 2.2992


In [9]:
y_pred = model.predict(X_tests)

In [10]:
print("Model Scores: ")
print("r2 score: ",r2_score(y_pred,y_test))
print("mse: ",mean_squared_error(y_pred,y_test))

Model Scores: 
r2 score:  0.9774683566611484
mse:  2.3315356


In [11]:
y_test

,AverageTemperature
8492499,28.540001
2922609,-5.670000
3466259,7.992000
4853510,10.308000
4285403,19.714001
...,...
5280283,-15.023000
6838087,26.806999
3047911,19.278000
874359,23.396999


In [12]:
y_pred

array([[28.122658 ],
       [-4.8693714],
       [10.682406 ],
       ...,
       [19.772034 ],
       [24.110777 ],
       [-1.4669906]], dtype=float32)

In [21]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [22]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
      hp.Choice('units', [100,250,500]),
      activation='tanh',input_dim = 7))
    model.add(keras.layers.Dense(
      hp.Choice('units', [300,700,1200]),
      activation='tanh'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [300,700,1200]),
      activation='tanh'))
    model.add(keras.layers.Dense(1, activation='tanh'))
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer = Adam(learning_rate=hp_lr) ,loss='mean_squared_error',)
    return model

In [23]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [24]:
tuner.search(X_trains, y_train, epochs=5, validation_data=(X_tests, y_test))
model = tuner.get_best_models()[0]

INFO:tensorflow:Oracle triggered exit


ValueError: Shapes (7, 40) and (17, 40) are incompatible